### Initialize

In [ ]:
from datetime import datetime
import json
import os
import sys

from ipywidgets import Dropdown
import pandas as pd
import requests

sys.path.append(os.path.join(sys.path[0], '..', '..', '..', '..'))
from demo.shared.tools import TENANT_SUFFIX, get_base_url, get_root_path, print_response

base_url = get_base_url()
root_path = get_root_path()


### Select example which should be used

In [ ]:
example_path = os.path.join('..', '..','..','examples')
print(example_path)
options = next(os.walk(example_path, topdown=True))[1]
selected = Dropdown(
    options=options,
    description='Example:',
    value = None,
    disabled=False,
)
selected

### Create CSON (JSON-format) from CDS file (text format) using the CDS compiler

In [ ]:
from json import JSONDecodeError


if selected.value:
    cds_file = os.path.join(root_path, '..','..','..', 'examples', selected.value, 'model.cds')
    cson_file = os.path.join(root_path, '..', 'temp', f'{selected.value}.model.json')
    data_file = os.path.join(root_path, '..','..','..', 'examples', selected.value, 'data.json')
    command_line_statement = (f'cds compile {cds_file} --to json > {cson_file}')
    os.system(command_line_statement)
  
    with open(cson_file, encoding='utf-8') as f:
        cson = json.load(f)

    print(f'The following command line statement was executed to compile the example cds file to cson. Filename is temp{os.sep}{selected.value}.model.json):')
    print(command_line_statement)
else:
    print('No example selected')


### List existing tenants

In [ ]:
r = requests.get(f'{base_url}/v1/tenant')
if r.status_code == 200:
    existing_tenants = [{k:datetime.fromisoformat(v) if k == 'createdAt' else v 
        for k, v in w.items()} for w in r.json()]
    if existing_tenants:
        df = pd.DataFrame.from_dict(existing_tenants)
        df.rename(columns={'name': 'Tenant Name', 'createdAt': 'Created At'}, inplace=True)
        display(df)
    else:
        print('Info: No tenants exist')
else:
    print(f'Error. HTTP response status code {r.status_code}')

### Create tenant

In [ ]:
r = requests.post(f'{base_url}/v1/tenant/{selected.value}Example')
print_response(r)

### Check model consistency

In [ ]:
r = requests.post(f'{base_url}/v1/deploy/{selected.value}', json=cson\
                    , params={'simulate': True})
print_response(r)

### Deploy

In [ ]:
r = requests.post(f'{base_url}/v1/deploy/{selected.value}Example', json=cson)
print_response(r)


## Load data

In [ ]:
with open(data_file, encoding='utf-8') as p:
    data = json.load(p)

r = requests.post(f'{base_url}/v1/data/{selected.value}Example', json=data)
if r.status_code == 200:
    ids = r.json()
    if ids:
        r = requests.post(f'{base_url}/v1/read/{selected.value}Example', json= ids)
else:
    print("Error:", r.status_code)        